# 라이브러리 임포트

In [1]:
# google drive mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
## for from kobert_transformers.utils import get_tokenizer
!pip install kobert-transformers

In [3]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
!pip install mxnet # 파이썬에서 실행 안 됨
!pip install transformers
!pip install torch

In [4]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
#!pip install onnxruntime==1.8.0
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install boto3

fatal: destination path 'KoBERT' already exists and is not an empty directory.
/content/KoBERT


In [5]:
## for import gluonnlp as nlp
!pip install gluonnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661657 sha256=9d9ff027edb2f9348093d092e94b3796e5516a57eb2860c5bcc86a462d7945a7
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [7]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.31.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy

In [5]:
# import modules

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

import numpy as np

from tqdm import tqdm, tqdm_notebook
from kobert_transformers.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


In [7]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-z8lbtsnt/kobert-tokenizer_9902a13092ba44b3915457b47f5fbc95
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-z8lbtsnt/kobert-tokenizer_9902a13092ba44b3915457b47f5fbc95
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=65deeeacdbc3573dcc672114f59c9e89ce61bc44ec4fa4eeb0bb92fb0a99f4c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mj5fmhkc/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-wwvoab7p
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-wwvoa

In [8]:
# for 데이터프레임 내 non-string을 string으로 변환
import pandas as pd
from transformers import BertModel, BertTokenizer
from kobert_tokenizer import KoBERTTokenizer  # 1-3-2. BERTDataset 형식 변환 바로 밑 설치 코드 실행 후 실행하면 잘 돌아감

In [9]:
# 테스트 데이터셋 정의
class BERTDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        tokens = self.tokenizer(sentence)
        token_ids = tokens["input_ids"][:self.max_len]
        valid_length = len(token_ids)
        segment_ids = [0] * valid_length

        # Padding
        while len(token_ids) < self.max_len:
            token_ids.append(0)
            segment_ids.append(0)

        return (
            torch.tensor(token_ids, dtype=torch.long),
            torch.tensor(valid_length, dtype=torch.long),
            torch.tensor(segment_ids, dtype=torch.long),
            torch.tensor(label, dtype=torch.long),
        )

In [27]:
# 데이터 로드
# sentences = ["이 문장은 보이스피싱입니다.", "이 문장은 정상입니다."]
# labels = [1, 0]  # 1: 보이스피싱, 0: 정상
sentences = [
    "이 문장은 보이스피싱입니다.",
    "이 문장은 정상입니다.",
    "긴급히 연락 부탁드립니다. 은행 계좌 정보 필요합니다.",
    "안녕하세요, 새로운 이벤트에 초대됩니다.",
    "귀하의 카드 정보가 유출되었습니다. 확인 필요합니다.",
    "오늘 날씨가 좋습니다. 산책하기 좋은 날이에요.",
    "이 제품은 최고입니다. 지금 구매하세요!"
]
labels = [1, 0, 1, 0, 1, 0, 0]  # 1: 보이스피싱, 0: 정상

max_len = 64

tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")

test_dataset = BERTDataset(sentences, labels, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=1)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# ko-bert 기본 모델 정확도 평가

In [16]:
# BERT 모델과 Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()
MAX_LEN = 64

/content/KoBERT/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [13]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        # TorchScript 호환 방식으로 변경
        batch_size, seq_len = token_ids.size()
        attention_mask = torch.arange(seq_len, device=token_ids.device).expand(batch_size, seq_len)
        attention_mask = (attention_mask < valid_length.unsqueeze(1)).float()
        return attention_mask

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(
            input_ids=token_ids,
            token_type_ids=segment_ids.long(),
            attention_mask=attention_mask.float().to(token_ids.device),
            return_dict=False,
        )
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# 학습된 모델 로드
model_path = "/content/gdrive/MyDrive/crimefin/model/model.pt"
state_dict = torch.load(model_path, map_location=device)
bert_state_dict = state_dict['bert']
trained_model = BERTClassifier(bertmodel, num_classes=2, dr_rate=0.5).to(device)
trained_model.load_state_dict(bert_state_dict, strict=False)
trained_model.eval()


<ipython-input-17-caf69283d44d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)


BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [28]:
# Ko-BERT 기본 모델 평가 함수
def evaluate_bert(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for token_ids, valid_length, segment_ids, labels in dataloader:
            token_ids = token_ids.to(device)
            valid_length = valid_length.to(device)
            segment_ids = segment_ids.to(device)
            labels = labels.to(device)

            outputs = model(token_ids, valid_length, segment_ids)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy

In [29]:
# Ko-BERT 정확도 계산
accuracy_bert = evaluate_bert(trained_model, test_loader, device)
print(f"Ko-BERT 모델 정확도: {accuracy_bert:.4f}")

Ko-BERT 모델 정확도: 0.8571


# torchscript 모델 정확도 평가

In [19]:
# TorchScript 모델 로드
scripted_model = torch.jit.load("/content/gdrive/MyDrive/crimefin/model/torchscript_kobert.pt")
scripted_model.eval()

RecursiveScriptModule(
  original_name=BERTClassifier
  (bert): RecursiveScriptModule(
    original_name=BertModel
    (embeddings): RecursiveScriptModule(
      original_name=BertEmbeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (encoder): RecursiveScriptModule(
      original_name=BertEncoder
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=BertLayer
          (attention): RecursiveScriptModule(
            original_name=BertAttention
            (self): RecursiveScriptModule(
              original_name=BertSdpaSelfAttention
              (query): RecursiveScriptModule(original_name=

In [30]:
# TorchScript 모델 정확도 계산
accuracy_torchscript = evaluate_bert(scripted_model, test_loader, device)
print(f"TorchScript 모델 정확도: {accuracy_torchscript:.4f}")


TorchScript 모델 정확도: 0.8571


# onnx 모델 정확도 평가

In [22]:
!pip install onnxruntime
import onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00


In [23]:
# ONNX 모델 로드
onnx_model_path = "/content/gdrive/MyDrive/crimefin/model/onnx_kobert.onnx"
onnx_session = onnxruntime.InferenceSession(onnx_model_path)

In [24]:
# ONNX 평가 함수
def evaluate_onnx(session, dataloader):
    correct = 0
    total = 0

    for token_ids, valid_length, segment_ids, labels in dataloader:
        input_feed = {
            "input_ids": token_ids.numpy(),
            "valid_length": valid_length.numpy(),
            "segment_ids": segment_ids.numpy(),
        }

        outputs = session.run(None, input_feed)
        predictions = np.argmax(outputs[0], axis=1)
        correct += (predictions == labels.numpy()).sum()
        total += labels.size(0)

    accuracy = correct / total
    return accuracy

In [31]:
# ONNX 모델 정확도 계산
accuracy_onnx = evaluate_onnx(onnx_session, test_loader)
print(f"ONNX 모델 정확도: {accuracy_onnx:.4f}")

ONNX 모델 정확도: 0.8571


# 정확도 비교

In [26]:
print(f"Ko-BERT 기본 모델 정확도: {accuracy_bert:.4f}")
print(f"TorchScript 모델 정확도: {accuracy_torchscript:.4f}")
print(f"ONNX 모델 정확도: {accuracy_onnx:.4f}")

if accuracy_bert == accuracy_torchscript == accuracy_onnx:
    print("모든 모델의 정확도가 동일합니다.")
else:
    print("모델별 정확도 차이가 있습니다.")


Ko-BERT 기본 모델 정확도: 1.0000
TorchScript 모델 정확도: 1.0000
ONNX 모델 정확도: 1.0000
모든 모델의 정확도가 동일합니다.
